In [32]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [33]:
import sys

ABSOLUTE_PATH = "/content/drive/My Drive/deepfake_tweets"

PREPROCESSED_DATA_DIR = ABSOLUTE_PATH + "/data/preprocessed"
BERT_MODEL_DIR        = ABSOLUTE_PATH + "/models/bert-finetuned"
DISTILBERT_MODEL_DIR  = ABSOLUTE_PATH + "/models/distilbert-finetuned"
ROBERTA_MODEL_DIR     = ABSOLUTE_PATH + "/models/roberta-finetuned"

sys.path.insert(0, ABSOLUTE_PATH)

# Error Analysis

In [34]:
import os

import pandas as pd

import torch
from torch.utils.data import DataLoader
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
)

from utils import TweetDataset, compute_metrics, plot_confusion_matrix

In [35]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

### Load the evaluation dataset

In [59]:
test_df = pd.read_csv(os.path.join(PREPROCESSED_DATA_DIR, "tweepfake_test.csv"))

X_test = test_df['text'].values
y_test = (test_df['account.type'] == 'bot').astype(int).values

X_test, y_test

(array(['justin timberlake really one of the goats if you think about it',
        'Thank you <MENTION> for your gracious prayers and wishes from the people of Bhutan. We will work together to take this relationship to greater heights of peace and prosperity. <URL>',
        'Theory: the number of red lights you will hit while driving is directly proportional to how bad you have to urinate',
        ..., 'JOIN OUR TEAM: Sneezing',
        'These deeply discounted 256GB SanDisk flash drives give you SSD-sized storage on the go | PCWorld <URL> via <MENTION>',
        '<MENTION> diamond pearl and platinum were honestly good too'],
       dtype=object),
 array([0, 0, 0, ..., 1, 0, 0]))

In [58]:
pd.crosstab(test_df['account.type'], y_test)  # human = 0, bot = 0

col_0,0,1
account.type,,
bot,0,1280
human,1278,0


In [52]:
label_map = { 0: "human", 1: "bot" }

def get_error_analysis(preds):
  analysis_df = test_df.copy()
  analysis_df['true_label'] = pd.Series(y_test, index=analysis_df.index).map(label_map)
  analysis_df['predicted_label'] = pd.Series(preds, index=analysis_df.index).map(label_map)
  analysis_df['correct'] = analysis_df['true_label'] == analysis_df['predicted_label']
  return analysis_df


args = TrainingArguments(report_to=["tensorboard"])

## BERT (bert-base-cased)

### Load model and tokenizer

In [38]:
bert_model = AutoModelForSequenceClassification.from_pretrained(BERT_MODEL_DIR)
bert_tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL_DIR)
bert_model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [40]:
bert_trainer = Trainer(model=bert_model, args=args)
bert_eval_dataset = TweetDataset(X_test, y_test, bert_tokenizer, 512)

In [41]:
bert_predictions = bert_trainer.predict(bert_eval_dataset)
bert_predicted_labels = bert_predictions.predictions.argmax(axis=-1)

In [55]:
bert_logits = bert_predictions.predictions
bert_pred_labels = bert_logits.argmax(axis=-1)

results_df = test_df.copy()
results_df['true_label']      = y_test
results_df['predicted_label'] = bert_pred_labels
results_df['correct']         = results_df['true_label'] == results_df['predicted_label']
results_df.head()

,screen_name,text,account.type,class_type,true_label,predicted_label,correct
0,zawvrk,justin timberlake really one of the goats if y...,human,human,0,0,True
1,narendramodi,Thank you <MENTION> for your gracious prayers ...,human,human,0,0,True
2,ahadsheriff,Theory: the number of red lights you will hit ...,human,human,0,1,False
3,AINarendraModi,Respects on the Upt of the I good with the peo...,bot,rnn,1,0,False
4,kevinhooke,Might give the BASIC <HASHTAG> game contest an...,human,human,0,0,True


In [61]:
bert_eror_analysis_df = get_error_analysis(bert_pred_labels)
misclassified = bert_eror_analysis_df[~bert_eror_analysis_df['correct']]

misclassified.head(50)

,screen_name,text,account.type,class_type,true_label,predicted_label,correct
2,ahadsheriff,Theory: the number of red lights you will hit ...,human,human,human,bot,False
3,AINarendraModi,Respects on the Upt of the I good with the peo...,bot,rnn,bot,human,False
7,jaden,I Wanna See Some Rainbows Out There Boys,human,human,human,bot,False
11,dril_gpt2,"there is a ""war on cows"" going on right now, b...",bot,gpt2,bot,human,False
14,dril,i miss 100% of the shots i dont take mother fu...,human,human,human,bot,False
16,awhalefact,"<MENTION> i’m sorry my jokes killed you, i wha...",human,human,human,bot,False
20,dril,getting a 4k monitor so i can make posts,human,human,human,bot,False
33,zawvrk,certified hood classic,human,human,human,bot,False
36,dril,"i have created more jobs, than every Weed Smok...",human,human,human,bot,False
38,ninjasexparty,porn movie idea: An Inspector Balls,human,human,human,bot,False


## DistilBERT (distilbert-base-uncased)